<a href="https://colab.research.google.com/github/AlperYildirim1/Astroarchaeology/blob/main/Sunrise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install astropy

In [ ]:
import numpy as np
import math
from astropy.time import Time
from astropy.coordinates import get_sun, EarthLocation, AltAz
import astropy.units as u

OBSERVER_LAT = 32.245461
OBSERVER_LON = 35.210617
OBSERVER_ALT = 422


HORIZON_ALTITUDE = 1.39

YEAR_1 = -1250
YEAR_2 = 625
YEAR_3 = 2024

def find_solstice_sunrise_azimuth(year, location, horizon_alt):

    astro_year = year if year > 0 else year + 1
    solstice_approx_time = Time({'year': astro_year, 'month': 12, 'day': 21, 'hour': 0, 'minute': 0, 'second': 0},
                                format='ymdhms', scale='utc')
    times_to_check = solstice_approx_time + np.linspace(4 * 60, 8 * 60, 500) * u.minute
    observer_frame = AltAz(obstime=times_to_check, location=location)
    sun_local_positions = get_sun(times_to_check).transform_to(observer_frame)

    sunrise_indices = np.where((sun_local_positions.alt > horizon_alt*u.deg) & (np.roll(sun_local_positions.alt, 1) <= horizon_alt*u.deg))[0]

    if len(sunrise_indices) > 0:
        first_sunrise_index = sunrise_indices[0]
        sunrise_azimuth = sun_local_positions[first_sunrise_index].az
        return sunrise_azimuth.to_string(unit=u.deg, decimal=True, precision=4)
    else:
        return "Sunrise not found for this horizon."

if __name__ == "__main__":
    observer_location_object = EarthLocation(lat=OBSERVER_LAT * u.deg, lon=OBSERVER_LON * u.deg, height=OBSERVER_ALT * u.m)

    print(f"Using Observer Location: Lat {OBSERVER_LAT}, Lon {OBSERVER_LON}, Alt {OBSERVER_ALT}m")
    print(f"Using Mountain Horizon Angle: {HORIZON_ALTITUDE} degrees")
    print("="*75)

    azimuth1 = find_solstice_sunrise_azimuth(YEAR_1, observer_location_object, HORIZON_ALTITUDE)
    print(f"In {YEAR_1} (Exodus Era), sunrise azimuth was: {azimuth1}")

    azimuth2 = find_solstice_sunrise_azimuth(YEAR_2, observer_location_object, HORIZON_ALTITUDE)
    print(f"In {YEAR_2} (Quranic Era), sunrise azimuth was: {azimuth2}")

    azimuth3 = find_solstice_sunrise_azimuth(YEAR_3, observer_location_object, HORIZON_ALTITUDE)
    print(f"In {YEAR_3} (Modern Era), sunrise azimuth was: {azimuth3}")
    print("="*75)